In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as stats
import pandas as pd
from geopy.distance import distance

In [2]:
df = pd.read_csv('radio_merger_data.csv')

In [3]:
df['scaled_pop'] = df['population_target']/1000000
df['scaled_price'] = df['price']/1000000

In [4]:
df['buyer_loc'] = df[['buyer_lat', 'buyer_long']].apply(tuple, axis=1)
df['target_loc'] = df[['target_lat', 'target_long']].apply(tuple, axis=1)
df['distance_mi'] = df.apply(lambda row: distance(row['buyer_loc'], row['target_loc']).miles, axis=1)

In [5]:
df07 = df.loc[df['year'] == 2007]
df08 = df.loc[df['year'] == 2008]
years = [df07, df08]

In [6]:
Buy = ['year', 'buyer_id', 'buyer_lat', 'buyer_long', 'num_stations_buyer', 'corp_owner_buyer']
Target = ['target_id', 'target_lat', 'target_long', 'scaled_price', 'hhi_target', 'scaled_pop']

In [7]:
counterfac = [x[Buy].iloc[i].values.tolist() + x[Target].iloc[j].values.tolist() 
             for x in years for i in range(len(x) - 1) 
             for j in range(i + 1, len(x))]
counterfactuals = pd.DataFrame(counterfac, columns = Buy + Target)

In [8]:
counterfactuals['buyer_loc'] = counterfactuals[['buyer_lat', 'buyer_long']].apply(tuple, axis=1)
counterfactuals['target_loc'] = counterfactuals[['target_lat', 'target_long']].apply(tuple, axis=1)
counterfactuals['distance_mi'] = counterfactuals.apply(lambda row: distance(row['buyer_loc'], row['target_loc']).miles, axis=1)

In [ ]:
# def payoff(data, i, parameters):
#     '''
#     Args:
#         data: data used for calculation
#         i: used to iterate through rows
#         parameters: initial parameter estimates for alpha and beta
    
#     Returns:
#         f: the payoff to the merger
#     '''
#     # note no coef on the first term
#     alpha = parameters[0]
#     beta = parameters[1]
    
#     f = data['num_stations_buyer'].iloc[i] * data['scaled_pop'].iloc[i] + alpha * data['corp_owner_buyer'].iloc[i] * data['scaled_pop'].iloc[i] + beta * data['distance_mi'].iloc[i]
    
#     return(f)

In [9]:
def payoff(data, parameters):
    '''
    Args:
        data: data used for calculation
        parameters: initial parameter estimates for alpha and beta
    
    Returns:
        f: the payoff to the merger
    '''
    # note no coef on the first term
    alpha = parameters[0]
    beta = parameters[1]
        
    f = data['num_stations_buyer'] * data['scaled_pop'] + alpha * data['corp_owner_buyer'] * data['scaled_pop'] + beta * data['distance_mi']
    
    return(f)

In [10]:
params = (0.5, 0.5)

In [35]:
# actual 2007 payoffs
# for i in range(len(df07)):
#     actual7 = [payoff(data = df07, parameters = params)]

In [126]:
actual = pd.DataFrame(payoff(data=df, parameters = params), columns=['payoff'])
actual7 = pd.DataFrame(payoff(data = df07, parameters = params))
actual8 = pd.DataFrame(payoff(data = df08, parameters = params))
print(actual)

        payoff
0    77.628327
1    25.801074
2    22.240889
3   101.020980
4     5.388469
..         ...
94  507.992337
95   27.746666
96  313.242063
97   25.018253
98    9.198588

[99 rows x 1 columns]


In [128]:
df = pd.concat([df, actual], axis=1)
print(df)

TypeError: concat() got an unexpected keyword argument 'inplace'

In [19]:
# actual 2008 payoffs
# for i in range(len(df08)):
#     actual8 = [payoff(data = df08, parameters = params)]

In [47]:
len07 = int(((len(df07) * (len(df07) - 1))/2))
print(len07)
print(len(counterfactuals))

990
2421


In [116]:
counter = pd.DataFrame(payoff(data=counterfactuals, parameters=params), columns=['payoff'])
counter7 = pd.DataFrame(payoff(data=counterfactuals, parameters=params))
counter7 = counter7[counter7.index < 990]

In [117]:
counterfactuals = pd.concat([counterfactuals, counter], axis=1)

In [66]:
counter8 = pd.DataFrame(payoff(data=counterfactuals, parameters=params))
counter8 = counter8[counter8.index > 989]

In [ ]:
# counterfactual 2007 payoffs
# len07 = int(((len(df07) * (len(df07) - 1))/2))
# for i in range(0,len07):
#     counter7 = [payoff(data=counterfactuals, parameters=params)]

In [80]:
# counterfactual 2008 payoffs
# for i in range(len07, len(counterfactuals)):
#     counter8 = [payoff(data=counterfactuals, parameters=params)]

In [100]:
for x in [[actual7, counter7], [actual8, counter8]]:
    print(len(x[0]))

45
54


In [130]:
def objective123(actual, counter):
    '''
    A function that returns the maximum score estimator
    
    Args:
        actual7: actual payoffs from 2007
        actual8: actual payoffs from 2008
        counter7: counterfactual payoffs from 2007
        counter8: counterfactual payoffs from 2008
        
    Returns:
        score: maximum score estimator
    '''
    score = 0
    for x in [actual, counter]:
        for i in range(len(actual)):
            for j in range(len(counter)):
                actual_sum = actual.loc[actual['payoff'][i]] + actual.loc[actual['payoff'][i+1]]
                counter_sum = counter.loc[counter['payoff'][j]] + counter.loc[counter['payoff'][j-1]]
    for i in range(len(actual)):
        if actual_sum >= counter_sum:
            score =+ 1
        else:
            score =+0

    return(actual_sum)

#     for x in [[actual7, counter7], [actual8, counter8]]:
#         for i in range(len(x[0])):
#             for j in range(len(x[0])):
#                 if j > i:
#                     if x[0][i] + x[0][j] >= x[1][j, i] + x[1][i, (j - 1)]:

In [68]:
def objective(actual7, actual8, counter7, counter8):
    '''
    A function that returns the maximum score estimator
    
    Args:
        actual7: actual payoffs from 2007
        actual8: actual payoffs from 2008
        counter7: counterfactual payoffs from 2007
        counter8: counterfactual payoffs from 2008
        
    Returns:
        score: maximum score estimator
    '''
    score = [1 for x in [[actual7, counter7], [actual8, counter8]]
            for i in range(len(x[0]))
            for j in range(len(x[0]))
            if j > i if x[0][i] + x[0][j] >= x[1][j, i] + x[1][i, (j - 1)]]

    return(score)

In [131]:
results = opt.minimize(objective123(df, counterfactuals), params, method = 'Nelder-Mead', options = {'maxiter': 5000})
print("Without Transfers Results:", results)

KeyError: 0

In [132]:
def payofftransfers(data, parameters):
    '''
    Args:
        data: data used for calculation
        parameters: initial parameter estimates for delta, alpha, gamma, and beta
    
    Returns:
        f: the payoff to the merger
    '''
    delta = parameters[0]
    alpha = parameters[1]
    gamma = parameters[2]
    beta = parameters[3]
    
    f2 = gamma * data['num_stations_buyer'] * data['scaled_pop'] + alpha * data['corp_owner_buyer'] * data['scaled_pop'] + gamma * data['hhi_target'] + beta * data['distance_mi']
    
    return(f2)

In [133]:
params2 = (0.5, 0.5, 0.5, 0.5)

In [134]:
# actual 2007 payoffs
actual07 = [payofftransfers(data = df07, parameters = params2)]

# actual 2008 payoffs
actual08 = [payofftransfers(data = df08, parameters = params2)]

In [135]:
# counterfactual 2007 payoffs
counter07 = [payofftransfers(data=counterfactuals, parameters=params2)]
counter07 = counter7[counter7.index < 990]
# counterfactual 2008 payoffs
counter08 = [payofftransfers(data=counterfactuals, parameters=params2)]
counter08 = counter8[counter8.index > 989]

In [142]:
def objective2(self, actual07, actual08, counter07, counter08):
    '''
    A function that returns the maximum score estimator
    
    Args:
        actual07: actual payoffs from 2007
        actual08: actual payoffs from 2008
        counter07: counterfactual payoffs from 2007
        counter08: counterfactual payoffs from 2008
        
    Returns:
        score: maximum score estimator
    '''
    price07 = df07['scaled_price'].tolist()
    price08 = df08['scaled_price'].tolist()
    score = [1 for m in [[actual07, counter07, price07], [actual08, counter08, price08]]
            for i in range(len(m[0]))
            for j in range(len(m[0]))
            if (m[0][i] - m[1][j, i] >= m[2][i] - m[2][j]) & (m[0][j] - m[1][i, (j - 1)] >= m[2][j] - m[2][i])]
    
    return(score)

In [143]:
results = opt.minimize(objective2, params2, args = (actual07, actual08, counter07, counter08), method = 'Nelder-Mead', options = {'maxiter': 5000})
print("With Transfers Results:", results)

KeyError: (0, 0)